In [ ]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torch.nn.functional as F
import seaborn as sns
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("torch version:", torch.__version__)
print("device:", device)

torch version: 2.1.0+cu121
device: cuda


In [ ]:
destination_directory = '/content/gdrive/MyDrive/Physionet_NPY_DATA/'

In [ ]:
X_train_file_path = destination_directory + 'X_train.npy'
X_test_file_path = destination_directory + 'X_test.npy'
y_train_file_path = destination_directory + 'y_train.npy'
y_test_file_path = destination_directory + 'y_test.npy'

In [ ]:
X_train_file_path

'/content/gdrive/MyDrive/Physionet_NPY_DATA/X_train.npy'

In [ ]:
X_train = np.load(X_train_file_path, allow_pickle=True)
X_test = np.load(X_test_file_path, allow_pickle=True)
y_train = np.load(y_train_file_path, allow_pickle=True)
y_test = np.load(y_test_file_path, allow_pickle=True)

In [ ]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [ ]:
X_train = X_train[:10000,:,:]

In [ ]:
X_train.shape

(10000, 5000, 12)

In [ ]:
y_train = y_train[:10000]

In [ ]:
y_list = y_train.tolist()

temp = None
faulty_set = list()
count = 0
for items in y_train:

    if len(items[0]) == 0:
       count += 1
       temp = items
       faulty_set.append(temp)
       continue

X_train = X_train[np.where(y_train != temp)[0]]
y_train = y_train[np.where(y_train != temp)[0]]
X_test = X_test[np.where(y_test != temp)[0]]
y_test = y_test[np.where(y_test != temp)[0]]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9854, 5000, 12), (1901, 5000, 12), (9854, 1), (1901, 1))

In [ ]:
# shape of one ECG
np.array(X_train[0]).shape

(5000, 12)

In [ ]:

unique_values = np.unique(np.concatenate(y_train.flatten()))
print(f'All Labels: {unique_values}')

All Labels: ['CD' 'HYP' 'MI' 'NORM' 'STTC']


In [ ]:
x_out = list()
y_out = list()
to_pop = list()
sample_size = 1
count = 0
for i in y_train:
    if i[0] == ['CD']:
        x_out.append(X_train[count])
        y_out.append(y_train[count])
        to_pop.append(count)
        X_train = np.delete(X_train, count, axis=0)
        y_train = np.delete(y_train, count, axis=0)
        break
    count += 1

In [ ]:
np.array(x_out).shape, np.array(y_out).shape

((1, 5000, 12), (1, 1))

In [ ]:
x_out

[array([[-0.009,  0.144,  0.153, ..., -0.051, -0.099, -0.066],
        [-0.012,  0.192,  0.204, ..., -0.068, -0.132, -0.088],
        [-0.015,  0.24 ,  0.255, ..., -0.085, -0.165, -0.11 ],
        ...,
        [-0.005,  0.01 ,  0.015, ..., -0.03 ,  0.03 ,  0.02 ],
        [-0.004,  0.008,  0.012, ..., -0.024,  0.024,  0.016],
        [-0.003,  0.006,  0.009, ..., -0.018,  0.018,  0.012]])]

In [ ]:
y_out

[array([list(['CD'])], dtype=object)]

In [ ]:
import gc
gc.collect()

0

In [ ]:
y_out

[array([list(['CD'])], dtype=object)]

In [ ]:
count

24

In [ ]:
X_train.shape

(9853, 5000, 12)

In [ ]:
import pickle
with open('x_patient.pkl', 'wb') as file:
    pickle.dump(np.array(x_out), file)

with open('y_patient.pkl', 'wb') as file:
    pickle.dump(np.array(y_out), file)

# with open('new_X_train.pkl', 'wb') as file:
#     pickle.dump(np.array(X_train), file)

# with open('new_y_train.pkl', 'wb') as file:
#     pickle.dump(np.array(y_train), file)

In [ ]:
np.array(x_out).shape, np.array(y_out).shape

((1, 5000, 12), (1, 1))

In [ ]:
# category mapping to a number
num2class = np.array(['NORM', 'MI', 'STTC', 'CD', 'HYP'])

def label2num(labels):
    for index in range(len(labels)):
        labels[index] = np.where(num2class == labels[index][0][0])[0]

In [ ]:
# y_train_copy = y_train.copy()
# y_test_copy = y_test.copy()

In [ ]:
# label2num(y_train_copy)
# label2num(y_test_copy)

In [ ]:
label2num(y_train)
label2num(y_test)

In [ ]:
y_train = y_train.flatten().astype(np.int32)
y_test = y_test.flatten().astype(np.int32)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9853, 5000, 12), (1901, 5000, 12), (9853,), (1901,))

In [ ]:
batch_size = 16

In [ ]:
X_train.shape

(9853, 5000, 12)

In [ ]:

X_train = X_train.reshape(X_train.shape[0], 1, 5000, 12)
X_test = X_test.reshape(X_test.shape[0], 1, 5000, 12)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class BasicBlock(nn.Module):

    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels),
            )

    def forward(self, x):
        out = self.features(x)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

In [ ]:

basic_block = BasicBlock(64, 128)
print(basic_block)
x = torch.randn(2, 64, 500, 12)
y = basic_block(x)
print(y.shape)

BasicBlock(
  (features): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential(
    (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
torch.Size([2, 128, 500, 12])


In [ ]:
class Bottleneck(nn.Module):

    expansion = 4

    def __init__(self, in_channels, zip_channels, stride=1):
        super(Bottleneck, self).__init__()
        out_channels = self.expansion * zip_channels
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, zip_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(zip_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(zip_channels, zip_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(zip_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(zip_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.features(x)
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

In [ ]:
bottleneck = Bottleneck(256, 128)
print(bottleneck)
x = torch.randn(2, 256, 32, 32)
y = bottleneck(x)
print(y.shape)

Bottleneck(
  (features): Sequential(
    (0): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential(
    (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
torch.Size([2, 512, 32, 32])


In [ ]:
class ResNet(nn.Module):
    """
    ResNet architecture for processing input images. This class initializes the network with
    convolutional layers, followed by multiple residual blocks, average pooling, and a final
    fully connected layer for classification.

    The network architecture is designed to process input images and output class scores for
    a specified number of classes. It utilizes the concept of residual learning with either
    BasicBlock or Bottleneck blocks to enable deep network architectures.
    """

    def __init__(self, block, num_blocks, num_classes=5):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(7, 1), stride=(2, 1), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        # Initialize residual blocks
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Use adaptive average pooling to ensure the output size is (batch_size, channels, 1, 1)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # The classifier linear layer; adjust in_features based on the block expansion and the output of avg_pool
        self.classifier = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # Pass input through the initial features
        out = self.features(x)

        # Pass through each residual layer
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        # Apply adaptive average pooling and flatten the output
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)  # Flatten the output for the fully connected layer

        # Pass through the classifier to get class scores
        out = self.classifier(out)
        return out

In [ ]:
def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

In [ ]:
net = ResNet34().to(device)
print(net)
if device == 'cuda':
    net = nn.DataParallel(net)
    torch.backends.cudnn.benchmark = True

ResNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 1), stride=(2, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (features): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (features): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [ ]:
x = torch.randn(2, 1, 500, 12).to(device)
y = net(x)
print(y.shape)

torch.Size([2, 5])


Train Model

In [ ]:
lr = 1e-1
momentum = 0.9
weight_decay = 5e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold=0.1, patience=3, verbose=True)

In [ ]:
# training function
def train(epoch):
    print('\nEpoch: %d' % (epoch))
    net.train()
    train_loss = torch.Tensor([0.0]).float().to(device)
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets.long())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.6f |  Acc: %.3f%% (%d/%d)' %
              (epoch + 1, batch_idx + 1, train_loss.item(), 100.*correct/total, correct, total))
    return train_loss

In [ ]:
load_model = False
if load_model:
    checkpoint = torch.load('./checkpoint/res18.ckpt')
    net.load_state_dict(checkpoint['net'])
    start_epoch = checkpoint['epoch']
else:
    start_epoch = 0
print('start_epoch: %s' % start_epoch)

start_epoch: 0


In [ ]:
for epoch in range(start_epoch, 10):
    loss = train(epoch)
    print('Total loss: %.6f' % loss)
    start_epoch = epoch
    scheduler.step(loss)


Epoch: 0
[1,   100] loss: 213.106934 |  Acc: 44.875% (718/1600)
[1,   200] loss: 353.771484 |  Acc: 46.156% (1477/3200)
[1,   300] loss: 489.882629 |  Acc: 47.021% (2257/4800)
[1,   400] loss: 623.595764 |  Acc: 47.797% (3059/6400)
[1,   500] loss: 754.764587 |  Acc: 48.600% (3888/8000)
[1,   600] loss: 881.958862 |  Acc: 49.531% (4755/9600)
Total loss: 902.196594

Epoch: 1
[2,   100] loss: 118.565224 |  Acc: 56.688% (907/1600)
[2,   200] loss: 235.388290 |  Acc: 57.094% (1827/3200)
[2,   300] loss: 353.819244 |  Acc: 57.104% (2741/4800)
[2,   400] loss: 461.776489 |  Acc: 58.266% (3729/6400)
[2,   500] loss: 569.367249 |  Acc: 59.237% (4739/8000)
[2,   600] loss: 676.128418 |  Acc: 59.719% (5733/9600)
Total loss: 693.427429

Epoch: 2
[3,   100] loss: 104.237671 |  Acc: 63.500% (1016/1600)
[3,   200] loss: 204.618088 |  Acc: 63.312% (2026/3200)
[3,   300] loss: 301.828674 |  Acc: 64.000% (3072/4800)
[3,   400] loss: 401.999268 |  Acc: 64.062% (4100/6400)
[3,   500] loss: 501.656860 | 

In [ ]:
torch.save(net.state_dict(), 'resnet.pth')

In [ ]:
import shutil

In [ ]:
shutil.copy('resnet.pth', '/content/gdrive/MyDrive/Physionet_NPY_DATA')

'/content/gdrive/MyDrive/Physionet_NPY_DATA/resnet.pth'

In [ ]:
with open('/content/x_patient.pkl', 'rb') as file:
    x_patient = pickle.load(file)

In [ ]:
with open('/content/y_patient.pkl', 'rb') as file:
    y_patient = pickle.load(file)

In [ ]:
x_patient.shape

(1, 5000, 12)

In [ ]:
import numpy as np

# Assuming arr is your NumPy array with shape (1, 5000, 12)
# arr = ...

# Batch size
batch_size = 500

# Calculate the number of batches
num_batches = x_patient.shape[1] // batch_size

# Reshape into batches
batches = np.array_split(x_patient, num_batches, axis=1)

# 'batches' is a list of arrays, each with shape (1, 500, 12)

# Accessing individual batches
for i, batch in enumerate(batches):
    print(f"Batch {i + 1} shape: {batch.shape}")
    # Do something with the batch


Batch 1 shape: (1, 500, 12)
Batch 2 shape: (1, 500, 12)
Batch 3 shape: (1, 500, 12)
Batch 4 shape: (1, 500, 12)
Batch 5 shape: (1, 500, 12)
Batch 6 shape: (1, 500, 12)
Batch 7 shape: (1, 500, 12)
Batch 8 shape: (1, 500, 12)
Batch 9 shape: (1, 500, 12)
Batch 10 shape: (1, 500, 12)


In [ ]:
X_train = np.reshape(np.array(batches), (10, 500, 12))

In [ ]:
y_train = np.load(y_train_file_path, allow_pickle=True)
y_train = y_train.reshape(len(y_train), 1)

y_list = y_train.tolist()

temp = None
faulty_set = list()
count = 0
for items in y_train:

    if len(items[0]) == 0:
       count += 1
       temp = items
       faulty_set.append(temp)
       continue




In [ ]:
y_train.shape

(17302, 1)

In [ ]:

y_train = y_train[np.where(y_train != temp)[0]]

label2num(y_train)
y_train = y_train.flatten().astype(np.int32)

In [ ]:
y_train = y_train[24]

In [ ]:
y_train

3

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

In [ ]:
y_train = y_train.repeat(10, 1)

y_train = y_train.squeeze()

In [ ]:
X_train = X_train.unsqueeze(1)

X_train.shape, y_train.shape

(torch.Size([10, 1, 500, 12]), torch.Size([10]))

In [ ]:

batch_size = 1
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

In [ ]:
X_train.shape, y_train.shape

(torch.Size([10, 1, 500, 12]), torch.Size([10]))

In [ ]:
start_epoch = 0
for epoch in range(start_epoch, 3):
    loss = train(epoch)
    print('Total loss: %.6f' % loss)
    start_epoch = epoch
    scheduler.step(loss)


Epoch: 0
Total loss: 7.356345

Epoch: 1
Total loss: 0.297312

Epoch: 2
Total loss: 0.067956


In [ ]:
torch.save(net.state_dict(), 'resnet_finetuned.pth')
shutil.copy('resnet_finetuned.pth', '/content/gdrive/MyDrive/Physionet_NPY_DATA')

'/content/gdrive/MyDrive/Physionet_NPY_DATA/resnet_finetuned.pth'

In [ ]:
y_test

tensor([0., 0., 0.,  ..., 0., 3., 1.])

In [ ]:
y_train

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

In [ ]:
y_train = np.load(y_train_file_path, allow_pickle=True)
y_train = y_train.reshape(len(y_train), 1)

y_list = y_train.tolist()

temp = None
faulty_set = list()
count = 0
for items in y_train:

    if len(items[0]) == 0:
       count += 1
       temp = items
       faulty_set.append(temp)
       continue

y_train = y_train[np.where(y_train != temp)[0]]

label2num(y_train)
y_train = y_train.flatten().astype(np.int32)
y_train = torch.tensor(y_train, dtype=torch.float32)



In [ ]:
y_train.shape

torch.Size([16966])

In [ ]:
import numpy as np

def add_gaussian_noise(data, noise_level=0.05):
    """
    Adds Gaussian noise to the data.

    :param data: Original data array (1, time_points, channels).
    :param noise_level: Standard deviation of the Gaussian noise.
    :return: Data with added Gaussian noise.
    """
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

def random_time_shift(data, max_shift=100):
    """
    Randomly shifts the data in time.

    :param data: Original data array (1, time_points, channels).
    :param max_shift: Maximum number of time points to shift.
    :return: Time-shifted data.
    """
    shift = np.random.randint(-max_shift, max_shift)
    return np.roll(data, shift, axis=1)

def scale_amplitude(data, scale_range=(0.8, 1.2)):
    """
    Randomly scales the amplitude of the data.

    :param data: Original data array (1, time_points, channels).
    :param scale_range: Tuple indicating min and max scaling factors.
    :return: Amplitude-scaled data.
    """
    scale_factor = np.random.uniform(scale_range[0], scale_range[1])
    return data * scale_factor

# Simulate an ECG data sample
ecg_data_sample = np.random.rand(1, 500, 12)  # Your ECG data with shape (1, 5000, 12)

# Apply augmentations
augmented_data = add_gaussian_noise(np.array(batches))
augmented_data = random_time_shift(augmented_data)
augmented_data = scale_amplitude(augmented_data)

print("Original Data Shape:", ecg_data_sample.shape)
print("Augmented Data Shape:", augmented_data.shape)

Original Data Shape: (1, 500, 12)
Augmented Data Shape: (10, 1, 500, 12)


In [ ]:
net = ResNet34().to(device)
# chk = torch.load('resnet.pth')
# model.load_state_dict(chk['model_state_dict'])
state_dict = torch.load('/content/resnet.pth')

# Apply the weights
net.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
X_train = torch.tensor(augmented_data, dtype=torch.float32)

In [ ]:
batch_size = 1
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

In [ ]:
X_train.shape, y_train.shape

(torch.Size([10, 1, 500, 12]), torch.Size([10]))

In [ ]:

start_epoch = 0
for epoch in range(start_epoch, 3):
    loss = train(epoch)
    print('Total loss: %.6f' % loss)
    start_epoch = epoch
    scheduler.step(loss)


Epoch: 0
[1,     1] loss: 1.940569 |  Acc: 0.000% (0/1)
[1,     2] loss: 3.569745 |  Acc: 0.000% (0/2)
[1,     3] loss: 5.028925 |  Acc: 0.000% (0/3)
[1,     4] loss: 6.610187 |  Acc: 0.000% (0/4)
[1,     5] loss: 7.994543 |  Acc: 0.000% (0/5)
[1,     6] loss: 10.178669 |  Acc: 0.000% (0/6)
[1,     7] loss: 11.622543 |  Acc: 0.000% (0/7)
[1,     8] loss: 13.881487 |  Acc: 0.000% (0/8)
[1,     9] loss: 15.400972 |  Acc: 0.000% (0/9)
[1,    10] loss: 17.116020 |  Acc: 0.000% (0/10)
Total loss: 17.116020

Epoch: 1
[2,     1] loss: 1.715048 |  Acc: 0.000% (0/1)
[2,     2] loss: 3.296310 |  Acc: 0.000% (0/2)
[2,     3] loss: 5.555253 |  Acc: 0.000% (0/3)
[2,     4] loss: 7.074739 |  Acc: 0.000% (0/4)
[2,     5] loss: 9.258865 |  Acc: 0.000% (0/5)
[2,     6] loss: 10.718045 |  Acc: 0.000% (0/6)
[2,     7] loss: 12.161920 |  Acc: 0.000% (0/7)
[2,     8] loss: 13.546276 |  Acc: 0.000% (0/8)
[2,     9] loss: 15.175452 |  Acc: 0.000% (0/9)
[2,    10] loss: 17.116020 |  Acc: 0.000% (0/10)
Total 